In [4]:
import pandas as pd
import numpy as np

import configparser



In [6]:
import os
data_path = os.path.abspath("../data/raw/DataLoans_0424.csv")

In [7]:
dataset = pd.read_csv(data_path, delimiter = ";")
dataset.head()

,id,first_name,last_name,email,gender,Desembolso,Vencimiento,Tasa Original,Tasa Actual,Fecha Último Incremento,...,TIR,Cant.Pagarés,Suma_Asegurada,Flujo_Ingreso,Fujo_Costo,Com_Desembolso,Opex,PE,Rec_Capital,ISR
0,1,Celeste,Gallafant,cgallafant0@google.cn,Female,2/08/2022,30/04/2037,6.75,7.50,31/03/2024,...,0.006062,0,0.000,2.020633e+06,-1.694724e+06,0.0,0.0,97795.48672,3724668.653,-140440.66620
1,2,Alex,Skunes,askunes1@clickbank.net,Female,12/09/2022,28/02/2039,7.50,7.75,28/02/2023,...,0.008112,0,1511406.916,1.857647e+06,-1.509339e+06,0.0,0.0,98768.73707,2902574.199,-128877.29420
2,3,Kev,Enns,kenns2@ft.com,Male,23/10/2023,25/05/2026,7.25,7.50,31/03/2024,...,0.005582,0,0.000,3.088080e+05,-2.509065e+05,0.0,0.0,50038.62688,2573399.796,-21462.15430
3,4,Sioux,Di Meo,sdimeo3@xinhuanet.com,Female,17/07/2023,17/07/2028,8.25,8.50,31/03/2024,...,0.009979,0,0.000,4.828663e+05,-3.692507e+05,0.0,0.0,12103.22912,2272312.020,-33491.83332
4,5,Caron,Niave,cniave4@businessweek.com,Female,23/11/2023,25/05/2026,7.00,7.50,31/03/2024,...,0.005594,0,0.000,2.048941e+05,-1.664764e+05,0.0,0.0,33200.62893,1707450.764,-14188.18243


In [142]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

dataset['SegmentoComercial'] = label_encoder.fit_transform(dataset['SegmentoComercial'])

dataset.head()

,id,first_name,last_name,email,gender,Desembolso,Vencimiento,Tasa Original,Tasa Actual,Fecha Último Incremento,...,TIR,Cant.Pagarés,Suma_Asegurada,Flujo_Ingreso,Fujo_Costo,Com_Desembolso,Opex,PE,Rec_Capital,ISR
0,1,Celeste,Gallafant,cgallafant0@google.cn,Female,2/08/2022,30/04/2037,6.75,7.50,31/03/2024,...,0.006062,0,0.000,2.020633e+06,-1.694724e+06,0.0,0.0,97795.48672,3724668.653,-140440.66620
1,2,Alex,Skunes,askunes1@clickbank.net,Female,12/09/2022,28/02/2039,7.50,7.75,28/02/2023,...,0.008112,0,1511406.916,1.857647e+06,-1.509339e+06,0.0,0.0,98768.73707,2902574.199,-128877.29420
2,3,Kev,Enns,kenns2@ft.com,Male,23/10/2023,25/05/2026,7.25,7.50,31/03/2024,...,0.005582,0,0.000,3.088080e+05,-2.509065e+05,0.0,0.0,50038.62688,2573399.796,-21462.15430
3,4,Sioux,Di Meo,sdimeo3@xinhuanet.com,Female,17/07/2023,17/07/2028,8.25,8.50,31/03/2024,...,0.009979,0,0.000,4.828663e+05,-3.692507e+05,0.0,0.0,12103.22912,2272312.020,-33491.83332
4,5,Caron,Niave,cniave4@businessweek.com,Female,23/11/2023,25/05/2026,7.00,7.50,31/03/2024,...,0.005594,0,0.000,2.048941e+05,-1.664764e+05,0.0,0.0,33200.62893,1707450.764,-14188.18243


In [143]:
config = configparser.ConfigParser()
config.read('../pipeline.cfg')

#valor = config.get('GENERAL','TARGET')

['../pipeline.cfg']

In [144]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.encoding import OrdinalEncoder
from feature_engine.outliers import Winsorizer
from feature_engine.creation import MathFeatures

In [145]:
drop_vars = list(config.get('GENERAL','VARS_TO_DROP').split(','))
drop_vars.append(config.get('GENERAL','TARGET'))

x_features = dataset.drop(labels = drop_vars, axis = 1)
y_target =dataset[config.get('GENERAL','TARGET')]
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = 0.3, shuffle = True, random_state = 2025)

In [146]:
# Pipeline
loans_model = Pipeline([
    # Frequency Encoding
    ('freq_var_encoding', CountFrequencyEncoder(
        encoding_method='count', 
        variables=config.get('ENCODING', 'VARS_TO_FREQ_ENCODE').split(',')
    )),

    # Label Encoding
    ('lab_var_encoding', OrdinalEncoder(
        encoding_method='ordered', 
        variables=config.get('ENCODING', 'VARS_TO_LABEL_ENCODE').split(',')
    )),

    # Método Capping
    ('capping_var', Winsorizer(
        capping_method='iqr',
        tail='both',
        fold=1.5,
        variables=config.get('CAPPING', 'VARS_TO_CAPP').split(',')
    )),

    # Transformación polinomial (cuadrática)
    #('pol_var', MathFeatures(
        #variables=config.get('TRANSFORM', 'VARS_TO_QUADRATIC').split(','),  # Variables a transformar
        #func=['power'] * len(config.get('TRANSFORM', 'VARS_TO_QUADRATIC').split(',')),  # Aplicar 'power' a cada variable
        #new_variables_names=config.get('TRANSFORM', 'NEW_VARS_QUADRATIC').split(',')  # Nuevos nombres
    #)),

    # Feature Scaling
    ('feature_scaling', StandardScaler())
])

In [147]:
loans_model.fit(x_train,y_train)

Pipeline(steps=[('freq_var_encoding',
                 CountFrequencyEncoder(variables=['Código Moneda',
                                                  'Status Mora', 'Nuevo'])),
                ('lab_var_encoding',
                 OrdinalEncoder(variables=['Brecha_Plazo'])),
                ('capping_var',
                 Winsorizer(capping_method='iqr', fold=1.5, tail='both',
                            variables=['Saldo$', 'PE_30.04.2024',
                                       'PlazoRestante(Dias)',
                                       'Reserva Especifica$', 'Ab_24',
                                       'Tasa_Propuesta',
                                       'IntAnual_TasaPropuesta', 'FTP',
                                       'TIR'])),
                ('feature_scaling', StandardScaler())])

In [148]:
x_features_processed = loans_model.transform(x_train)
df_features_processed = pd.DataFrame(x_features_processed, columns = x_train.columns)
df_features_processed['SegmentoComercial'] = y_train.reset_index()['SegmentoComercial']

df_features_processed.to_csv('../data/processed/feature_for_models.csv',index=False)

In [149]:
x_features_processed_test = loans_model.transform(x_test)
df_features_processed_test = pd.DataFrame(x_features_processed_test, columns = x_test.columns)
df_features_processed_test['SegmentoComercial'] = y_test.reset_index()['SegmentoComercial']

df_features_processed_test.to_csv('../data/processed/test_dataset.csv',index=False)

In [151]:
import pickle

with open('../artifacts/pipeline.pkl','wb') as f:
    pickle.dump(loans_model,f)